In [68]:
import torch
from torch import nn
from torch.nn.utils import rnn
import torch.nn.functional as F
import pandas as pd
import copy
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
# from torchsummary import summary
from torchinfo import summary
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vectors,GloVe
from collections import Counter


In [7]:
vocab_size = 30000
glove = GloVe(name='6B', dim=300)

.vector_cache/glove.6B.zip: 862MB [02:44, 5.24MB/s]                               
100%|█████████▉| 399999/400000 [01:08<00:00, 5829.78it/s]


In [8]:
print(type(glove))

<class 'torchtext.vocab.vectors.GloVe'>


In [9]:
glove['hello']

tensor([-3.3712e-01, -2.1691e-01, -6.6365e-03, -4.1625e-01, -1.2555e+00,
        -2.8466e-02, -7.2195e-01, -5.2887e-01,  7.2085e-03,  3.1997e-01,
         2.9425e-02, -1.3236e-02,  4.3511e-01,  2.5716e-01,  3.8995e-01,
        -1.1968e-01,  1.5035e-01,  4.4762e-01,  2.8407e-01,  4.9339e-01,
         6.2826e-01,  2.2888e-01, -4.0385e-01,  2.7364e-02,  7.3679e-03,
         1.3995e-01,  2.3346e-01,  6.8122e-02,  4.8422e-01, -1.9578e-02,
        -5.4751e-01, -5.4983e-01, -3.4091e-02,  8.0017e-03, -4.3065e-01,
        -1.8969e-02, -8.5670e-02, -8.1123e-01, -2.1080e-01,  3.7784e-01,
        -3.5046e-01,  1.3684e-01, -5.5661e-01,  1.6835e-01, -2.2952e-01,
        -1.6184e-01,  6.7345e-01, -4.6597e-01, -3.1834e-02, -2.6037e-01,
        -1.7797e-01,  1.9436e-02,  1.0727e-01,  6.6534e-01, -3.4836e-01,
         4.7833e-02,  1.6440e-01,  1.4088e-01,  1.9204e-01, -3.5009e-01,
         2.6236e-01,  1.7626e-01, -3.1367e-01,  1.1709e-01,  2.0378e-01,
         6.1775e-01,  4.9075e-01, -7.5210e-02, -1.1

In [11]:
tokenizer = get_tokenizer('basic_english')

In [34]:
print(glove.vectors.shape)
print(type(glove.vectors))

torch.Size([400000, 300])
<class 'torch.Tensor'>


In [18]:
df = pd.read_csv('Data/train.csv')
sentences = df['text'].values
labels = df['label'].values

In [25]:
word_list = []
for sentence in sentences:
    for word in tokenizer(sentence):
        word_list.append(word)

print(len(word_list))
        

3345810


In [26]:
freq = Counter(word_list)
freq_ = sorted(freq,key=freq.get,reverse=True)

print(len(freq_))

56069


In [87]:
glove.stoi['this']
test = copy.deepcopy(glove.vectors[:240000])
print(test.shape)

torch.Size([240000, 300])


In [88]:
class BaseModel(nn.Module):
    '''
    input_size -> text vocab size
    '''
    def __init__(self, input_size, output_size, embedding_dim, hidden_dim, num_layers, batch_first):
        super(BaseModel, self).__init__()

        self.num_layers = num_layers
        self.batch_first = batch_first   
        self.hidden_dim = hidden_dim

        """
        TODO: Implement your own model. You can change the model architecture.
        """
        # self.embedding = nn.Embedding.from_pretrained(test,freeze=True)
        self.embedding = nn.Embedding(input_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=batch_first)
        self.fc = nn.Linear(hidden_dim, output_size)

    # the size of x in forward is (seq_length, batch_size) if batch_first=False
    def forward(self, x, input_lengths):
        batch_size = x.size(0) if self.batch_first else x.size(1)

        #h_0: (num_layers * num_directions, batch_size, hidden_size)
        h_0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim)
        c_0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim)

        embedding = self.embedding(x)
        packed_input = pack_padded_sequence(embedding, input_lengths.tolist(), batch_first=self.batch_first)
        # output, hidden = self.rnn(packed_input, (h_0, c_0))
        packed_output, hidden = self.rnn(packed_input)
        # output, _ = pad_packed_sequence(output, batch_first=self.batch_first)

        output = self.fc(hidden[0][-1])

        # outputs, hidden = self.rnn(embedding, None)  # outputs.shape -> (sequence length, batch size, hidden size)

        # outputs = outputs[:, -1, :] if self.batch_first else outputs[-1, :, :]
        # output = self.fc(outputs)
        
        return output, hidden

In [89]:
model = BaseModel(240000, 2, 300, 64, 3, True)

In [90]:
model.embedding.weight.data.copy_(test)

tensor([[ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
        [-0.2554, -0.2572,  0.1317,  ..., -0.2329, -0.1223,  0.3550],
        [-0.1256,  0.0136,  0.1031,  ..., -0.3422, -0.0224,  0.1368],
        ...,
        [ 0.2044, -0.2027, -0.1458,  ...,  0.8147,  0.0636,  0.4498],
        [-0.3197, -0.7546,  0.7908,  ...,  0.1304, -0.1798, -0.1267],
        [ 1.0571,  0.1381,  0.1052,  ...,  0.0510,  0.6684, -0.6450]])

In [91]:
torch.save(model.state_dict(),'test.pt')